# Modeling

There are three steps to creating this model:

1. **Vectorization**
2. **Train/Validation/Test Split**
3. **Modeling**: We apply a baseline CNN.

In [67]:
import numpy as np
import re
import keras
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer
from keras import layers
from keras.models import Sequential
from keras.layers.embeddings import Embedding
from keras.layers import Dense, Conv1D, LSTM, GlobalMaxPooling1D, InputLayer, Dropout, SpatialDropout1D, Bidirectional
from keras.layers.normalization import BatchNormalization
from keras import optimizers
from keras import models
from keras import losses
from keras import metrics
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

# Import custom functions
from explore import *
from evaluation import *

In [3]:
# Import the dataset
df = pd.read_csv('processed_data/clean_data.csv')

In [6]:
df.isna().sum()

Tweet                       1830
Years                          0
Lemmatized                  3220
Tweets with no Stopwords    6230
Short Tweets                6893
dtype: int64

In [7]:
df = df.dropna()

In [24]:
print(df['Years'].unique())

[2018 2017 2019]


# CNN for Text Analysis

Neural networks analyze texts in a slightly different way with words as opposed to the sparse TF-IDF framework. Since this is a large dataset, a CNN maybe able to pick up intricate patterns. Preprocessing with CNNs requires it to be processed with Keras' `Embedding.()` when it comes to the modeling.

In [18]:
X_train, X_test, y_train, y_test = train_test_split(df['Tweets with no Stopwords'], 
                                                    df['Years'], test_size = 0.3, random_state = 2)

In [20]:
token = Tokenizer(num_words = 5000, filters = '!"#$%&()*+,-./:;<=>?@[\]^_`{"}~\t\n', lower = True, 
                  split = ' ', oov_token = True)
token.fit_on_texts(X_train)
X_train_seq = token.texts_to_sequences(X_train)
X_test_seq = token.texts_to_sequences(X_test)

In [28]:
# One hot encoding for labels
encoder = LabelBinarizer()
encoder.fit(y_train)
transformed = encoder.transform(y_train)
y_train_encoded = pd.DataFrame(transformed)
transformed_test = encoder.transform(y_test)
y_test_encoded = pd.DataFrame(transformed_test)

In [21]:
max_seq_length(X_train_seq)

83

In [29]:
max_length = 83

In [22]:
# Total number of words in the corpus
vocabulary_size = len(token.word_index)
vocabulary_size

374556

In [23]:
number_of_words = 5000

In [30]:
X_train_seq_pad = pad_sequences(X_train_seq, maxlen = max_length, padding = 'post')
X_test_seq_pad = pad_sequences(X_test_seq, maxlen = max_length, padding = 'post')

In [31]:
X_train_emb, X_val_emb, y_train_emb, y_val_emb = train_test_split(X_train_seq_pad, y_train_encoded, 
                                                                  test_size = 0.3, random_state = 3)

In [34]:
emb_model = models.Sequential()
emb_model.add(layers.Embedding(number_of_words, 200, input_length = max_length))
emb_model.add(layers.Conv1D(50, 3, activation = 'relu', input_shape = (200, 1)))
emb_model.add(layers.GlobalMaxPooling1D())
emb_model.add(layers.Dense(3, activation = 'softmax'))
emb_model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])

In [35]:
emb_model_history = emb_model.fit(X_train_emb, y_train_emb, epochs = 10, batch_size = 64, validation_split = 0.3)

Epoch 1/10
5582/5582 [==============================] - 234s 42ms/step - loss: 0.4772 - accuracy: 0.8313 - val_loss: 0.4219 - val_accuracy: 0.8487
Epoch 2/10
5582/5582 [==============================] - 235s 42ms/step - loss: 0.3918 - accuracy: 0.8549 - val_loss: 0.4318 - val_accuracy: 0.8429
Epoch 3/10
5582/5582 [==============================] - 233s 42ms/step - loss: 0.3493 - accuracy: 0.8673 - val_loss: 0.4351 - val_accuracy: 0.8432
Epoch 4/10
5582/5582 [==============================] - 233s 42ms/step - loss: 0.3002 - accuracy: 0.8863 - val_loss: 0.4724 - val_accuracy: 0.8313
Epoch 5/10
5582/5582 [==============================] - 233s 42ms/step - loss: 0.2548 - accuracy: 0.9045 - val_loss: 0.5157 - val_accuracy: 0.8279
Epoch 6/10
5582/5582 [==============================] - 230s 41ms/step - loss: 0.2140 - accuracy: 0.9216 - val_loss: 0.5731 - val_accuracy: 0.8266
Epoch 7/10
5582/5582 [==============================] - 234s 42ms/step - loss: 0.1803 - accuracy: 0.9344 - val_loss: 0

In [36]:
emb_model.evaluate(X_val_emb, y_val_emb)

6834/6834 [==============================] - 35s 5ms/step - loss: 0.8500 - accuracy: 0.8045


[0.8499530553817749, 0.8045334219932556]

In [40]:
emb_model.evaluate(X_test_seq_pad, y_test_encoded)

9763/9763 [==============================] - 50s 5ms/step - loss: 0.8505 - accuracy: 0.8045


[0.8504638075828552, 0.8045485019683838]

In [45]:
# Get classification report
y_pred = emb_model.predict(X_test_seq_pad, batch_size = 64, verbose = 1)
y_pred_bool = encoder.inverse_transform(y_pred) # Undo one-hot encoding
print(classification_report(y_test, y_pred_bool))

4882/4882 [==============================] - 39s 8ms/step
              precision    recall  f1-score   support

        2017       0.86      0.90      0.88     85882
        2018       0.84      0.85      0.84    191654
        2019       0.39      0.34      0.36     34874

    accuracy                           0.80    312410
   macro avg       0.70      0.69      0.70    312410
weighted avg       0.80      0.80      0.80    312410



In [46]:
emb_model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 83, 200)           1000000   
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 81, 50)            30050     
_________________________________________________________________
global_max_pooling1d_1 (Glob (None, 50)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 3)                 153       
Total params: 1,030,203
Trainable params: 1,030,203
Non-trainable params: 0
_________________________________________________________________


# FastText

In [55]:
def make_fast_text():
    fast_text = Sequential()
    fast_text.add(InputLayer((max_length,)))
    fast_text.add(Embedding(input_dim = vocabulary_size + 1, output_dim = 3, trainable = True))
    fast_text.add(SpatialDropout1D(0.5))
    fast_text.add(GlobalMaxPooling1D())
    fast_text.add(Dropout(0.5))
    fast_text.add(Dense(3, activation = 'softmax'))
    return fast_text

In [60]:
fast_text = make_fast_text()
fast_text.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
fast_text.fit(X_train_emb, y_train_emb, epochs = 10, verbose = 1, validation_split = 0.3)

Epoch 1/10
11163/11163 [==============================] - 223s 20ms/step - loss: 0.7227 - accuracy: 0.7083 - val_loss: 0.5698 - val_accuracy: 0.8287
Epoch 2/10
11163/11163 [==============================] - 212s 19ms/step - loss: 0.7201 - accuracy: 0.7094 - val_loss: 0.5683 - val_accuracy: 0.8295
Epoch 3/10
11163/11163 [==============================] - 203s 18ms/step - loss: 0.7226 - accuracy: 0.7085 - val_loss: 0.5727 - val_accuracy: 0.8292
Epoch 4/10
11163/11163 [==============================] - 202s 18ms/step - loss: 0.7211 - accuracy: 0.7091 - val_loss: 0.5789 - val_accuracy: 0.8203
Epoch 5/10
11163/11163 [==============================] - 193s 17ms/step - loss: 0.7193 - accuracy: 0.7104 - val_loss: 0.5721 - val_accuracy: 0.8265
Epoch 6/10
11163/11163 [==============================] - 213s 19ms/step - loss: 0.7189 - accuracy: 0.7107 - val_loss: 0.5732 - val_accuracy: 0.8258
Epoch 7/10
11163/11163 [==============================] - 240s 22ms/step - loss: 0.7193 - accuracy: 0.7106

In [61]:
# Get classification report
y_pred = fast_text.predict(X_test_seq_pad, batch_size = 64, verbose = 1)
y_pred_bool = encoder.inverse_transform(y_pred) # Undo one-hot encoding
print(classification_report(y_test, y_pred_bool))

4882/4882 [==============================] - 5s 1ms/step
              precision    recall  f1-score   support

        2017       0.85      0.89      0.87     85882
        2018       0.81      0.94      0.87    191654
        2019       0.00      0.00      0.00     34874

    accuracy                           0.82    312410
   macro avg       0.55      0.61      0.58    312410
weighted avg       0.73      0.82      0.77    312410



In [62]:
fast_text.summary()

Model: "sequential_17"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_16 (Embedding)     (None, 83, 3)             1123671   
_________________________________________________________________
spatial_dropout1d_14 (Spatia (None, 83, 3)             0         
_________________________________________________________________
global_max_pooling1d_16 (Glo (None, 3)                 0         
_________________________________________________________________
dropout_14 (Dropout)         (None, 3)                 0         
_________________________________________________________________
dense_16 (Dense)             (None, 3)                 12        
Total params: 1,123,683
Trainable params: 1,123,683
Non-trainable params: 0
_________________________________________________________________


# LSTM 

In [68]:
def define_lstm_model():
    model_lstm = Sequential()
    model_lstm.add(InputLayer((max_length,)))
    model_lstm.add(Embedding(input_dim = vocabulary_size + 1, output_dim = 3, trainable = True))
    model_lstm.add(SpatialDropout1D(0.5))
    model_lstm.add(BatchNormalization())
    model_lstm.add(Bidirectional(LSTM(125)))
    model_lstm.add(BatchNormalization())
    model_lstm.add(Dropout(0.5))
    model_lstm.add(Dense(3, activation = 'softmax'))
    return model_lstm

In [ ]:
model_lstm = define_lstm_model()
model_lstm.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
model_lstm.fit(X_train_emb, y_train_emb, validation_data = (X_val_emb, y_val_emb), epochs = 10, verbose = 1)

Epoch 1/10
  526/15946 [..............................] - ETA: 28:13 - loss: 0.9994 - accuracy: 0.6021

In [ ]:
# Get classification report
y_pred = model_lstm.predict(X_test_seq_pad, batch_size = 64, verbose = 1)
y_pred_bool = encoder.inverse_transform(y_pred) # Undo one-hot encoding
print(classification_report(y_test, y_pred_bool))